In [4]:
# import libraries
import time

import pandas as pd
import numpy as np
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By

In [1]:
url = "https://www.vrbo.com/search?adults=2&children=&regionId=122251&destination=Kitty%20Hawk%2C%20North%20Carolina%2C%20United%20States%20of%20America&privacyTrackingState=CAN_TRACK&searchId=033135fc-c70d-4793-aab6-8c1a158f2030&sort=RECOMMENDED&useRewards=SHOP_WITHOUT_POINTS&theme=&pwaDialog=&semdtl=&isInvalidatedDate=false&userIntent="

In [2]:
from vrbo_tool import main

df = main(url)

This is the page number:  1
This is the page number:  2
This is the page number:  3
This is the page number:  4
Finished collecting ulrs, collected 147


ValueError: All arrays must be of the same length

In [13]:
df = pd.read_csv("vrbo_links_old.csv")
df = df.iloc[9:, 1:]
df = df.reset_index(drop=True)
df

,links
0,https://www.vrbo.com/2632787?dateless=true&x_p...
1,https://www.vrbo.com/9864882ha?dateless=true&x...
2,https://www.vrbo.com/3325136?dateless=true&x_p...
3,https://www.vrbo.com/3195090?dateless=true&x_p...
4,https://www.vrbo.com/1719413?dateless=true&x_p...
...,...
468,https://www.vrbo.com/9908530ha?dateless=true&x...
469,https://www.vrbo.com/9379928ha?dateless=true&x...
470,https://www.vrbo.com/9798903ha?dateless=true&x...
471,https://www.vrbo.com/3406794?dateless=true&x_p...


In [15]:
driver = Chrome()

In [ ]:
# Define lists
urls_list = []
titels_list = []
prices_list = []

bedrooms_list = []
bathrooms_list = []
sleeps_list = []

popular_amenitie_0_list = []
popular_amenitie_1_list = []
popular_amenitie_2_list = []
popular_amenitie_3_list = []
popular_amenitie_4_list = []
popular_amenitie_5_list = []

# first drop un nessacry links
urls = df
url_num = 1

for index in range(122, len(urls)):  # Iterate over all links
    url = urls.iloc[index].values[0]
    urls_list.append(url)  # Add the url to the data_frame

    driver.get(url)
    time.sleep(3)

    try:
        title = driver.find_elements(
            By.XPATH, "//*[@class = 'uitk-heading uitk-heading-3']"
        )[0].text
        titels_list.append(title)  # The title of the listing
        print(f"Scraped page number {index}")
    except:
        titels_list.append(np.nan)
        print(f"Falied to scrape page number {index}")
    try:
        price = driver.find_element(By.XPATH, "//*[@class = 'uitk-lockup-price']").text
        prices_list.append(price)  # The price of the listing
    except:
        prices_list.append(np.nan)  # Error handling

    # Items list
    items_3 = driver.find_elements(
        By.XPATH,
        "//*[@class = 'uitk-text uitk-text-spacing-three uitk-type-300 uitk-text-standard-theme uitk-layout-flex-item uitk-layout-flex-item-flex-grow-1']",
    )

    items_4 = driver.find_elements(
        By.XPATH,
        "//*[@class = 'uitk-text uitk-text-spacing-three uitk-type-300 uitk-text-standard-theme uitk-layout-flex-item uitk-layout-flex-item-flex-basis-half_width uitk-layout-flex-item-flex-grow-1']",
    )

    if len(items_3) > 0:  # First if it has three items
        try:
            num_bedrooms = int(items_3[0].text.split()[0])
            bedrooms_list.append(num_bedrooms)
        except:
            bedrooms_list.append(np.nan)

        try:
            num_bathrooms = int(items_3[1].text.split()[0])
            bathrooms_list.append(num_bathrooms)
        except:
            bathrooms_list.append(np.nan)

        try:
            num_sleeps = int(items_3[2].text.split()[1])
            sleeps_list.append(num_sleeps)
        except:
            sleeps_list.append(np.nan)

    elif len(items_4) > 0:
        try:
            num_bedrooms = int(items_4[0].text.split()[0])
            bedrooms_list.append(num_bedrooms)
        except:
            bedrooms_list.append(np.nan)

        try:
            num_bathrooms = int(items_4[1].text.split()[0])
            bathrooms_list.append(num_bathrooms)
        except:
            bathrooms_list.append(np.nan)

        try:
            num_sleeps = int(items_4[2].text.split()[1])
            sleeps_list.append(num_sleeps)
        except:
            sleeps_list.append(np.nan)

    else:
        bedrooms_list.append(np.nan)
        bathrooms_list.append(np.nan)
        sleeps_list.append(np.nan)

    # Popular amenities
    try:
        # All the popular amenities in the listing
        popular_amenities = driver.find_elements(
            By.XPATH,
            "//*[@class = 'uitk-text uitk-type-300 uitk-text-default-theme uitk-layout-flex-item']",
        )

        for i in range(6):
            try:
                globals()[f"popular_amenitie_{i}"] = popular_amenities[i].text

            except:  # If there is less than 6 items
                globals()[f"popular_amenitie_{i}"] = np.nan

        # Add them to lists
        popular_amenitie_0_list.append(popular_amenitie_0)
        popular_amenitie_1_list.append(popular_amenitie_1)
        popular_amenitie_2_list.append(popular_amenitie_2)
        popular_amenitie_3_list.append(popular_amenitie_3)
        popular_amenitie_4_list.append(popular_amenitie_4)
        popular_amenitie_5_list.append(popular_amenitie_5)

    except:  # Error handling if no popular amenities at all
        popular_amenitie_0_list.append(np.nan)
        popular_amenitie_1_list.append(np.nan)
        popular_amenitie_2_list.append(np.nan)
        popular_amenitie_3_list.append(np.nan)
        popular_amenitie_4_list.append(np.nan)
        popular_amenitie_5_list.append(np.nan)

# Create the dataframe
all_data_df = pd.DataFrame(
    {
        "urls": urls_list,
        "title": titels_list,
        "prices": prices_list,
        "num_bedrooms": bedrooms_list,
        "num_bathrooms": bathrooms_list,
        "num_sleep": sleeps_list,
        "popular_amenitie_0": popular_amenitie_0_list,
        "popular_amenitie_1": popular_amenitie_1_list,
        "popular_amenitie_2": popular_amenitie_2_list,
        "popular_amenitie_3": popular_amenitie_3_list,
        "popular_amenitie_4": popular_amenitie_4_list,
        "popular_amenitie_5": popular_amenitie_5_list,
    }
)

IndexError: list index out of range

In [38]:
for index in range(122, len(urls)):  # Iterate over all links
    url = urls.iloc[index].values[0]
    urls_list.append(url)  # Add the url to the data_frame

    driver.get(url)
    time.sleep(3)

    try:
        title = driver.find_elements(
            By.XPATH, "//*[@class = 'uitk-heading uitk-heading-3']"
        )[0].text
        titels_list.append(title)  # The title of the listing
        print(f"Scraped page number {index}")
    except:
        titels_list.append(np.nan)
        print(f"Falied to scrape page number {index}")
    try:
        price = driver.find_element(By.XPATH, "//*[@class = 'uitk-lockup-price']").text
        prices_list.append(price)  # The price of the listing
    except:
        prices_list.append(np.nan)  # Error handling

    # Items list
    items_3 = driver.find_elements(
        By.XPATH,
        "//*[@class = 'uitk-text uitk-text-spacing-three uitk-type-300 uitk-text-standard-theme uitk-layout-flex-item uitk-layout-flex-item-flex-grow-1']",
    )

    items_4 = driver.find_elements(
        By.XPATH,
        "//*[@class = 'uitk-text uitk-text-spacing-three uitk-type-300 uitk-text-standard-theme uitk-layout-flex-item uitk-layout-flex-item-flex-basis-half_width uitk-layout-flex-item-flex-grow-1']",
    )

    if len(items_3) > 0:  # First if it has three items
        try:
            num_bedrooms = int(items_3[0].text.split()[0])
            bedrooms_list.append(num_bedrooms)
        except:
            bedrooms_list.append(np.nan)

        try:
            num_bathrooms = int(items_3[1].text.split()[0])
            bathrooms_list.append(num_bathrooms)
        except:
            bathrooms_list.append(np.nan)

        try:
            num_sleeps = int(items_3[2].text.split()[1])
            sleeps_list.append(num_sleeps)
        except:
            sleeps_list.append(np.nan)

    elif len(items_4) > 0:
        try:
            num_bedrooms = int(items_4[0].text.split()[0])
            bedrooms_list.append(num_bedrooms)
        except:
            bedrooms_list.append(np.nan)

        try:
            num_bathrooms = int(items_4[1].text.split()[0])
            bathrooms_list.append(num_bathrooms)
        except:
            bathrooms_list.append(np.nan)

        try:
            num_sleeps = int(items_4[2].text.split()[1])
            sleeps_list.append(num_sleeps)
        except:
            sleeps_list.append(np.nan)

    else:
        bedrooms_list.append(np.nan)
        bathrooms_list.append(np.nan)
        sleeps_list.append(np.nan)

    # Popular amenities
    try:
        # All the popular amenities in the listing
        popular_amenities = driver.find_elements(
            By.XPATH,
            "//*[@class = 'uitk-text uitk-type-300 uitk-text-default-theme uitk-layout-flex-item']",
        )

        for i in range(6):
            try:
                globals()[f"popular_amenitie_{i}"] = popular_amenities[i].text

            except:  # If there is less than 6 items
                globals()[f"popular_amenitie_{i}"] = np.nan

        # Add them to lists
        popular_amenitie_0_list.append(popular_amenitie_0)
        popular_amenitie_1_list.append(popular_amenitie_1)
        popular_amenitie_2_list.append(popular_amenitie_2)
        popular_amenitie_3_list.append(popular_amenitie_3)
        popular_amenitie_4_list.append(popular_amenitie_4)
        popular_amenitie_5_list.append(popular_amenitie_5)

    except:  # Error handling if no popular amenities at all
        popular_amenitie_0_list.append(np.nan)
        popular_amenitie_1_list.append(np.nan)
        popular_amenitie_2_list.append(np.nan)
        popular_amenitie_3_list.append(np.nan)
        popular_amenitie_4_list.append(np.nan)
        popular_amenitie_5_list.append(np.nan)

# Create the dataframe
all_data_df = pd.DataFrame(
    {
        "urls": urls_list,
        "title": titels_list,
        "prices": prices_list,
        "num_bedrooms": bedrooms_list,
        "num_bathrooms": bathrooms_list,
        "num_sleep": sleeps_list,
        "popular_amenitie_0": popular_amenitie_0_list,
        "popular_amenitie_1": popular_amenitie_1_list,
        "popular_amenitie_2": popular_amenitie_2_list,
        "popular_amenitie_3": popular_amenitie_3_list,
        "popular_amenitie_4": popular_amenitie_4_list,
        "popular_amenitie_5": popular_amenitie_5_list,
    }
)

Falied to scrape page number 122
Scraped page number 123
Scraped page number 124
Scraped page number 125
Scraped page number 126
Scraped page number 127
Scraped page number 128
Scraped page number 129
Scraped page number 130
Scraped page number 131
Scraped page number 132
Scraped page number 133
Scraped page number 134
Scraped page number 135
Scraped page number 136
Scraped page number 137
Scraped page number 138
Scraped page number 139
Scraped page number 140
Scraped page number 141
Scraped page number 142
Scraped page number 143
Scraped page number 144
Scraped page number 145
Scraped page number 146
Scraped page number 147
Scraped page number 148
Scraped page number 149
Scraped page number 150
Scraped page number 151
Scraped page number 152
Scraped page number 153
Scraped page number 154
Scraped page number 155
Scraped page number 156
Scraped page number 157
Scraped page number 158
Scraped page number 159
Scraped page number 160
Scraped page number 161
Scraped page number 162
Scraped

In [39]:
len(urls_list)

473

In [35]:
len(titels_list)

122

In [41]:
all_data_df.to_csv("all_links_results.csv")

In [23]:
len(sleeps_list)

10

In [16]:
from vrbo_tool import scrape_urls

scrape_urls(df, driver)

ValueError: All arrays must be of the same length